In [1]:
!pip install polars bottleneck

In [2]:
import polars as pl
import pandas as pd
import bottleneck as bn


1 чтение Titanic с помощью Polars

In [7]:
# Чтение train.csv с помощью Polars
titanic_pl = pl.read_csv("train.csv")

titanic_pl.head()


PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Owen Harris""","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. John Bradley (Fl…","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Miss. Laina""","""female""",26.0,0,0,"""STON/O2. 3101282""",7.925,null,"""S"""
4,1,1,"""Futrelle, Mrs. Jacques Heath (…","""female""",35.0,1,0,"""113803""",53.1,"""C123""","""S"""
5,0,3,"""Allen, Mr. William Henry""","""male""",35.0,0,0,"""373450""",8.05,null,"""S"""


2 основная информация о датасете (Polars)

In [8]:
print("Форма датафрейма (строки, столбцы):", titanic_pl.shape)

print("\nТипы данных столбцов:")
print(titanic_pl.dtypes)

print("\nОписательная статистика (describe):")
print(titanic_pl.describe())

print("\nЧисло пропусков в каждом столбце:")
print(titanic_pl.null_count())

print("\nСредние значения числовых столбцов:")
print(titanic_pl.select(pl.all().mean()))


Форма датафрейма (строки, столбцы): (891, 12)

Типы данных столбцов:
[Int64, Int64, Int64, String, String, Float64, Int64, Int64, String, Float64, String, String]

Описательная статистика (describe):
shape: (9, 13)
┌────────────┬─────────────┬──────────┬──────────┬───┬───────────┬───────────┬───────┬──────────┐
│ statistic  ┆ PassengerId ┆ Survived ┆ Pclass   ┆ … ┆ Ticket    ┆ Fare      ┆ Cabin ┆ Embarked │
│ ---        ┆ ---         ┆ ---      ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---   ┆ ---      │
│ str        ┆ f64         ┆ f64      ┆ f64      ┆   ┆ str       ┆ f64       ┆ str   ┆ str      │
╞════════════╪═════════════╪══════════╪══════════╪═══╪═══════════╪═══════════╪═══════╪══════════╡
│ count      ┆ 891.0       ┆ 891.0    ┆ 891.0    ┆ … ┆ 891       ┆ 891.0     ┆ 204   ┆ 889      │
│ null_count ┆ 0.0         ┆ 0.0      ┆ 0.0      ┆ … ┆ 0         ┆ 0.0       ┆ 687   ┆ 2        │
│ mean       ┆ 446.0       ┆ 0.383838 ┆ 2.308642 ┆ … ┆ null      ┆ 32.204208 ┆ null  ┆ null     │
│

3 количество пассажиров каждого класса (Pclass)

In [9]:
pclass_counts = (
    titanic_pl
    .get_column("Pclass")
    .value_counts()
    .sort("Pclass")
)

print(pclass_counts)


shape: (3, 2)
┌────────┬───────┐
│ Pclass ┆ count │
│ ---    ┆ ---   │
│ i64    ┆ u32   │
╞════════╪═══════╡
│ 1      ┆ 216   │
│ 2      ┆ 184   │
│ 3      ┆ 491   │
└────────┴───────┘


4 количество выживших мужчин и женщин

In [11]:
survived_by_sex = (
    titanic_pl
    .group_by("Sex")
    .agg(
        pl.col("Survived").sum().alias("Survived_count")
    )
)

survived_by_sex



Sex,Survived_count
str,i64
"""male""",109
"""female""",233


5 пассажиры старше 44 лет

In [12]:
older_44 = titanic_pl.filter(pl.col("Age") > 44)

older_44.head()


PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
7,0,1,"""McCarthy, Mr. Timothy J""","""male""",54.0,0,0,"""17463""",51.8625,"""E46""","""S"""
12,1,1,"""Bonnell, Miss. Elizabeth""","""female""",58.0,0,0,"""113783""",26.55,"""C103""","""S"""
16,1,2,"""Hewlett, Mrs. (Mary D Kingcome…","""female""",55.0,0,0,"""248706""",16.0,null,"""S"""
34,0,2,"""Wheadon, Mr. Edward H""","""male""",66.0,0,0,"""C.A. 24579""",10.5,null,"""S"""
53,1,1,"""Harper, Mrs. Henry Sleeper (My…","""female""",49.0,1,0,"""PC 17572""",76.7292,"""D33""","""C"""


# Ускорение работы с pandas + bottleneck
1 чтение Titanic с помощью pandas

In [13]:
titanic_pd = pd.read_csv("train.csv")
titanic_pd.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


2 средний возраст и стандартное отклонение с помощью bottleneck

In [14]:
age_values = titanic_pd["Age"].values  # NumPy-массив с возможными NaN

age_mean = bn.nanmean(age_values)
age_std = bn.nanstd(age_values)

print(f"Средний возраст (bn.nanmean): {age_mean:.2f}")
print(f"Стандартное отклонение (bn.nanstd): {age_std:.2f}")


Средний возраст (bn.nanmean): 29.70
Стандартное отклонение (bn.nanstd): 14.52


3 новый столбец Fare_new (через itertuples)

In [15]:
fare_new = []

for row in titanic_pd.itertuples():
    # row.Fare – значение тарифа у текущего пассажира
    fare_new.append(row.Fare * 1.3)

titanic_pd["Fare_new"] = fare_new

titanic_pd[["Fare", "Fare_new"]].head()


,Fare,Fare_new
0,7.2500,9.42500
1,71.2833,92.66829
2,7.9250,10.30250
3,53.1000,69.03000
4,8.0500,10.46500


# Оптимизация типов pandas на Housing.csv
чтение Housing.csv и исходная информация

In [16]:
housing = pd.read_csv("Housing.csv")

print("Исходная информация о датафрейме:")
housing.info(memory_usage="deep")


Исходная информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 221.9 KB


анализ типов

In [20]:
print("\nЧисло уникальных значений в столбцах:")
print(housing.nunique())

print("\nПример первых строк:")
housing.head()



Число уникальных значений в столбцах:
price               219
area                284
bedrooms              6
bathrooms             4
stories               4
mainroad              2
guestroom             2
basement              2
hotwaterheating       2
airconditioning       2
parking               4
prefarea              2
furnishingstatus      3
dtype: int64

Пример первых строк:


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [18]:
print("\nТипы данных столбцов:")
print(housing.dtypes)

print("\nПодробная информация о датафрейме:")
housing.info()



Типы данных столбцов:
price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object

Подробная информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwat

Анализ исходного датафрейма показал:

Число строк: 545

Число столбцов: 13

Числовые признаки (price, area, bedrooms, bathrooms, stories, parking) представлены как int64

Остальные признаки — строковые (object), при этом большинство содержит 2–3 уникальных значения

На основе этого оптимальные типы данных следующие:

1. price -> int32
Целочисленный признак. Значения цены значительно меньше предела int32, поэтому использование int64 избыточно.

2. area -> int32
Площадь — целые числа, диапазон также укладывается в int32. Это уменьшает потребление памяти вдвое.

3. bedrooms, bathrooms, stories, parking -> int8
Эти признаки имеют очень маленькое количество уникальных значений (от 2 до 6).
Диапазон int8 (−128…127) полностью покрывает реальные значения, что позволяет уменьшить память с 8 байт на значение до 1 байта.

4. mainroad, guestroom, basement, hotwaterheating, airconditioning, prefarea, furnishingstatus -> category
Это категориальные признаки с малым числом уникальных значений.
Тип category хранит их как словарь категорий + числовые коды вместо строк, что даёт существенную экономию памяти по сравнению с типом object.

оптимизации типов и сравнения памяти

In [19]:
import pandas as pd

# Копия исходного датафрейма
df_opt = housing.copy()

# Память до оптимизации
memory_before = df_opt.memory_usage(deep=True).sum() / 1024**2
print(f"Память до оптимизации: {memory_before:.3f} MB\n")

print("Исходные типы:")
print(df_opt.dtypes)

# 1. Оптимизация числовых столбцов
df_opt["price"] = df_opt["price"].astype("int32")
df_opt["area"] = df_opt["area"].astype("int32")

df_opt["bedrooms"] = df_opt["bedrooms"].astype("int8")
df_opt["bathrooms"] = df_opt["bathrooms"].astype("int8")
df_opt["stories"] = df_opt["stories"].astype("int8")
df_opt["parking"] = df_opt["parking"].astype("int8")

# 2. Оптимизация строковых столбцов -> category
cat_cols = [
    "mainroad",
    "guestroom",
    "basement",
    "hotwaterheating",
    "airconditioning",
    "prefarea",
    "furnishingstatus",
]

for col in cat_cols:
    df_opt[col] = df_opt[col].astype("category")

print("\nТипы после оптимизации:")
print(df_opt.dtypes)

# Память после оптимизации
memory_after = df_opt.memory_usage(deep=True).sum() / 1024**2
print(f"\nПамять после оптимизации: {memory_after:.3f} MB")
print(f"Экономия памяти: {memory_before - memory_after:.3f} MB")


Память до оптимизации: 0.217 MB

Исходные типы:
price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object

Типы после оптимизации:
price                  int32
area                   int32
bedrooms                int8
bathrooms               int8
stories                 int8
mainroad            category
guestroom           category
basement            category
hotwaterheating     category
airconditioning     category
parking                 int8
prefarea            category
furnishingstatus    category
dtype: object

Память после оптимизации: 0.011 MB
Экономия памяти: 0.205 MB
